In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/Users/dengxiaoyan/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_s = pd.read_excel('df_romney_serology_anti_s.xlsx')
data_n = pd.read_excel('df_romney_serology_anti_n.xlsx')

age = pd.read_excel('AgeSexData.xlsx')
age.columns = ['ID','Type','Sex','Age']
#age = age[age['ID']!= 146]
age = age[age['ID']!= 114]
age = age[age['ID']!= 125]
age = age[age['ID']!= 50]#lack of data
#age = age[age['ID']!= 79]#outlier : second measure <third measure
#age = age[age['ID']!= 49]#outlier : second measure <third measure
age = age[age['ID']!= 82]#outlier : extremely good initial dose
age = age[age['ID']!= 43]#outlier : only two neutralzation data
age = age[age['ID']!= 154]#outlier : too good response
age = age[age['ID']!= 134]#outlier : too good response
age = age[age['ID']!= 94]#outlier : high antibody but low neutralization
age = age[age['ID']!= 77]#outlier : weird neutralization antibody curve
antiS = data_s[['Study_ID_short', 'Visit','Result','Results description']]
antiS.columns =  ['ID','Visit','Result','Results description']
antiS.loc[antiS['Visit']  == 'B1','Visit'] = '6'
antiS.loc[antiS['Visit']  == 'B2','Visit'] = '7'
antiS.loc[antiS['Visit']  == 'B3','Visit'] = '8'
antiS.loc[antiS['Result']  == '>25000','Result'] = '25000'
antiS.loc[antiS['Result']  == '<0.4','Result'] = '0.4'
#antiS= antiS[antiS['ID']!= 146]
antiS= antiS[antiS['ID']!= 2]#outlier
antiS= antiS[antiS['ID']!= 114]
antiS= antiS[antiS['ID']!= 125]
#antiS= antiS[antiS['ID']!= 79]
#antiS= antiS[antiS['ID']!= 49]
antiS= antiS[antiS['ID']!= 50]
antiS= antiS[antiS['ID']!= 82]
antiS= antiS[antiS['ID']!= 43]
antiS= antiS[antiS['ID']!= 154]
antiS= antiS[antiS['ID']!= 134]
antiS= antiS[antiS['ID']!= 94]
antiS= antiS[antiS['ID']!= 77]
antiN = data_n[['Study_ID_short', 'Visit','Results description']]
antiN.columns =  ['ID','Visit','Results description']
antiN.loc[antiN['Visit']  == 'B1','Visit'] = '6'
antiN.loc[antiN['Visit']  == 'B2','Visit'] = '7'
antiN.loc[antiN['Visit']  == 'B3','Visit'] = '8'
#antiN= antiN[antiN['ID']!= 146]
antiS= antiS[antiS['ID']!= 2]#outlier
antiN= antiN[antiN['ID']!= 114]
antiN= antiN[antiN['ID']!= 125]
#antiN= antiN[antiN['ID']!= 49]
#antiN= antiN[antiN['ID']!= 79]
antiN= antiN[antiN['ID']!= 50]
antiN= antiN[antiN['ID']!= 82]
antiN= antiN[antiN['ID']!= 43]
antiN= antiN[antiN['ID']!= 154]
antiN= antiN[antiN['ID']!= 134]
antiN= antiN[antiN['ID']!= 94]
antiN= antiN[antiN['ID']!= 77]

In [3]:
spike_total = pd.DataFrame(columns = ['ID','Time','Result','Description_S','Description_N'])
spike_total['ID'] = antiS['ID']
spike_total['Time'] = antiS['Visit']
spike_total['Result'] = antiS['Result']
spike_total['Description_S'] = antiS['Results description']
spike_total['Description_N'] = antiN['Results description']

In [4]:
spike_total.loc[spike_total['Time']=='1','Time'] = 0
spike_total.loc[spike_total['Time']=='2','Time'] = 30 #30 after 1 dose
spike_total.loc[spike_total['Time']=='3','Time'] = 90 #2 months dosing interval and 30 after 2nd dose 2*30+60
spike_total.loc[spike_total['Time']=='4','Time'] = 150 #90 after 2nd dose
spike_total.loc[spike_total['Time']=='5','Time'] = 240 #+90 180 after 2nd dose
spike_total.loc[spike_total['Time']=='6','Time'] = 300 #3rd is 7 months after 2nd dose, 30 days after 3nd dose
spike_total.loc[spike_total['Time']=='7','Time'] = 360 #+60 90 after 3rd dose
spike_total.loc[spike_total['Time']=='8','Time'] = 450 #+90 180 after 3rd dose

In [5]:
spike_total['Description_S'] = spike_total['Description_S']
spike_total['Description_N'] = spike_total['Description_N']


In [6]:
spike_total.loc[spike_total['Description_N']=='Reactive','Result']=np.NaN
ids = spike_total['ID'].unique()
age_new = age[age['ID'].isin(ids)]
#spike_total.head(10)

In [7]:
a =  pd.unique(age_new['ID'])
b = np.repeat(a,10)
c = np.repeat(age_new['Type'],10)
d = np.repeat(age_new['Sex'],10)
e = np.repeat(age_new['Age'],10)
#f = np.repeat(0, len(b))
f = np.full((len(b),),np.nan)
dose = pd.DataFrame({'ID':b, 'Amount':(1,0,1,0,0,0,1,0,0,0)*len(a) ,'Time':(0,30,60,90,150,240,270,300,360,450)*len(a),
                     'Result':f,'Type':c, 'Sex':d,'Age':e})

In [8]:
for i in range(len(dose)):
    a = dose['ID'].iloc[i]
    dose.loc[(dose['ID'] == a)&(dose['Time'] == 30),'Result']=spike_total[(spike_total['ID'] == a)&(spike_total['Time'] == 30)]['Result'].to_string(index=False)
    dose.loc[(dose['ID'] == a)&(dose['Time'] == 90),'Result']=spike_total[(spike_total['ID'] == a)&(spike_total['Time'] == 90)]['Result'].to_string(index=False)
    dose.loc[(dose['ID'] == a)&(dose['Time'] == 150),'Result']=spike_total[(spike_total['ID'] == a)&(spike_total['Time'] == 150)]['Result'].to_string(index=False)
    dose.loc[(dose['ID'] == a)&(dose['Time'] == 240),'Result']=spike_total[(spike_total['ID'] == a)&(spike_total['Time'] == 240)]['Result'].to_string(index=False)
    dose.loc[(dose['ID'] == a)&(dose['Time'] == 300),'Result']=spike_total[(spike_total['ID'] == a)&(spike_total['Time'] == 300)]['Result'].to_string(index=False)
    dose.loc[(dose['ID'] == a)&(dose['Time'] == 360),'Result']=spike_total[(spike_total['ID'] == a)&(spike_total['Time'] == 360)]['Result'].to_string(index=False)
    dose.loc[(dose['ID'] == a)&(dose['Time'] == 450),'Result']=spike_total[(spike_total['ID'] == a)&(spike_total['Time'] == 450)]['Result'].to_string(index=False)


In [9]:
dose = dose.reset_index()

#plt.plot(dose['Time'],dose['Result'])

In [10]:
dose['Result'] = dose['Result'].astype(float)
index1 = dose[(dose['Time']== 30 )&(dose['Result'] > 2000)].index

dose.drop(index1,inplace = True)

In [11]:
index2 = dose[(dose['Time']== 150 )&(dose['Result'] > 20000)].index

dose.drop(index2,inplace = True)

In [12]:
index3 = dose[(dose['Amount']== 0)&(np.isnan(dose['Result']))].index
dose.drop(index3,inplace = True)


In [13]:
a = dose.groupby(by =['ID']).count()
idx = a['Result']<5 #drop the ID with less than 5 data points
b = a[idx]
c = b.index.tolist()
dose = dose[~dose['ID'].isin(c)]

dose2 = dose[(dose['Time']== 90)|(dose['Time']== 150)|(dose['Time']== 240)]
a = dose2.groupby(by =['ID']).count()
idx = a['Result']<1 #drop the ID with less than 1 data points
b = a[idx]
c = b.index.tolist()
dose_new1 = dose[~dose['ID'].isin(c)]

dose3 = dose[(dose['Time']== 300)|(dose['Time']== 360)|(dose['Time']== 450)]
a = dose3.groupby(by =['ID']).count()
idx = a['Result']<1 #drop the ID with less than 1 data points
b = a[idx]
c = b.index.tolist()
dose_new = dose_new1[~dose_new1['ID'].isin(c)]

dose_new.insert(2,'Censoring', 0)
dose_new.loc[(dose['Type']=='HCW')&(dose['Result'] > 20000),'Censoring'] = -1
dose_new.loc[(dose['Type']=='SENIOR')&(dose['Result'] == 25000),'Censoring'] = -1


In [14]:
hcw = dose_new[dose_new['Type']=='HCW']
senior = dose_new[dose_new['Type']=='SENIOR']

In [15]:
hcw = hcw[(hcw['Age']>34)&(hcw['Age']<52)]
#senior = senior[(senior['Age']>72)&(senior['Age']<84)]
cohort = pd.concat([hcw,senior])

In [16]:
dose_new = cohort

In [17]:
dose_new = dose_new[['ID','Amount','Censoring','Time','Result','Type','Age','Sex']]

In [18]:
dose_new.loc[(dose_new['Type'] == 'HCW')&(dose_new['Time'] == 270),'Amount']=0.5

/var/folders/wj/fd0ddm2j4gl8y60hhhv1mpn80000gn/T/ipykernel_19636/1300722039.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dose_new.loc[(dose_new['Type'] == 'HCW')&(dose_new['Time'] == 270),'Amount']=0.5


In [19]:
#plt.plot(dose_new['Time'],dose_new['Result'])

In [20]:
dose_data = dose_new[['ID','Amount','Censoring','Age','Time','Result','Type','Sex']]
dose_data.to_csv('all_monolix_age.csv', index=False)

In [21]:
senior= dose_new[dose_new['Type']=='SENIOR']
hcw = dose_new[dose_new['Type']=='HCW']
senior_data = senior[['ID','Amount','Censoring','Age','Time','Result','Type','Sex']]
hcw_data = hcw[['ID','Amount','Censoring','Age','Time','Result','Type','Sex']]

senior_data.to_csv('senior_monolix_age.csv', index=False)
hcw_data.to_csv('hcw_monolix_age.csv', index=False)